In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

<input>:23: DeprecationWarning: invalid escape sequence \S
<input>:26: DeprecationWarning: invalid escape sequence \s
<input>:23: DeprecationWarning: invalid escape sequence \S
<input>:26: DeprecationWarning: invalid escape sequence \s
<input>:23: DeprecationWarning: invalid escape sequence \S
<input>:26: DeprecationWarning: invalid escape sequence \s
<ipython-input-3-2b8fa5b210d4>:23: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-3-2b8fa5b210d4>:26: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']
['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 't

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


data_words = list(sent_to_words(data))

print(data_words[:1])


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])


# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]
['from', 'wheres', 'my', 'thi

In [7]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

[(0,
  '0.033*"lunar" + 0.026*"henry_spencer" + 0.024*"gate" + '
  '0.017*"toronto_zoology" + 0.015*"henry" + 0.014*"sam" + 0.010*"prize" + '
  '0.009*"mans_work" + 0.009*"kipling_utzoo" + 0.000*"tone"'),
 (1,
  '0.019*"galaxy" + 0.018*"steer" + 0.009*"newbie" + 0.007*"disappoint" + '
  '0.002*"how" + 0.000*"ariane" + 0.000*"mjs" + 0.000*"launch" + '
  '0.000*"countersteer" + 0.000*"leavitt"'),
 (2,
  '0.086*"fan" + 0.050*"motif" + 0.034*"mailing_list" + 0.031*"jeff" + '
  '0.030*"shipping" + 0.024*"window_manager" + 0.018*"socket" + 0.017*"suit" + '
  '0.016*"ltd" + 0.016*"exit"'),
 (3,
  '0.096*"key" + 0.035*"encryption" + 0.025*"clipper" + 0.025*"clipper_chip" + '
  '0.023*"algorithm" + 0.019*"security" + 0.019*"bit" + 0.018*"system" + '
  '0.017*"technology" + 0.017*"secure"'),
 (4,
  '0.080*"pittsburgh" + 0.060*"blood" + 0.034*"gordon_bank" + '
  '0.026*"surrender" + 0.024*"gordon_banks" + 0.019*"arrogance" + '
  '0.018*"chastity" + 0.017*"intellect" + 0.017*"shameful" + 0.017*"so

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.375998 -0.099498       1        1  40.678116
10    -0.320521 -0.080907       2        1  20.052551
16    -0.297536 -0.078330       3        1  10.708565
11    -0.205212 -0.016159       4        1   5.685738
8     -0.161423  0.025259       5        1   5.331521
12    -0.107832  0.460593       6        1   4.410882
14    -0.177682 -0.032583       7        1   3.750943
13    -0.139681  0.038267       8        1   3.618979
17     0.066459  0.040772       9        1   1.271769
3      0.019535 -0.040957      10        1   1.255503
7      0.133863 -0.007149      11        1   0.672793
18     0.140583 -0.005291      12        1   0.643736
5      0.166867 -0.019939      13        1   0.429547
9      0.152974 -0.016209      14        1   0.416192
2      0.175765 -0.023265      15        1   0.322899
15     0.180864 -0.027053      16        1   0.232194
19     0.180094 -0.025423      17        1   0.231216
4      0.175034 -0.029182      18        1   0.187595
0      0.194904 -0.030653      19        1   0.075619
1      0.198943 -0.032293      20        1   0.023634, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
4100   Default  60809.000000               ax  60809.000000  30.0000  30.0000
80     Default  24095.000000              not  24095.000000  29.0000  29.0000
21     Default  19489.000000             line  19489.000000  28.0000  28.0000
30     Default  17340.000000     organization  17340.000000  27.0000  27.0000
301    Default  10226.000000              say  10226.000000  26.0000  26.0000
3677   Default   4292.000000              max   4292.000000  25.0000  25.0000
48     Default   5366.000000             year   5366.000000  24.0000  24.0000
149    Default  13497.000000            would  13497.000000  23.0000  23.0000
63     Default  15136.000000               do  15136.000000  22.0000  22.0000
316    Default   7179.000000              use   7179.000000  21.0000  21.0000
201    Default   5161.000000           system   5161.000000  20.0000  20.0000
16     Default   6675.000000             host   6675.000000  19.0000  19.0000
410    Default   7474.000000           people   7474.000000  18.0000  18.0000
458    Default   2980.000000              god   2980.000000  17.0000  17.0000
1880   Default   3302.000000            space   3302.000000  16.0000  16.0000
45     Default   8012.000000       university   8012.000000  15.0000  15.0000
2695   Default   2798.000000             game   2798.000000  14.0000  14.0000
241    Default  11785.000000               be  11785.000000  13.0000  13.0000
561    Default   2656.000000             team   2656.000000  12.0000  12.0000
6      Default   1744.000000              car   1744.000000  11.0000  11.0000
333    Default   4096.000000            reply   4096.000000  10.0000  10.0000
126    Default   6643.000000              may   6643.000000   9.0000   9.0000
747    Default   1904.000000              key   1904.000000   8.0000   8.0000
522    Default   4904.000000             need   4904.000000   7.0000   7.0000
221    Default   8667.000000               go   8667.000000   6.0000   6.0000
947    Default   3311.000000          believe   3311.000000   5.0000   5.0000
672    Default   4385.000000            right   4385.000000   4.0000   4.0000
955    Default   2406.000000        christian   2406.000000   3.0000   3.0000
336    Default   3560.000000            state   3560.000000   2.0000   2.0000
106    Default   2493.000000              bit   2493.000000   1.0000   1.0000
...        ...           ...              ...           ...      ...      ...
6211   Topic20      0.004983              grb      1.051918   2.9979 -11.2274
18606  Topic20      0.004983       separation      1.051931   2.9979 -11.2274
6644   Topic20      0.004983     motorc

In [9]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import matplotlib.colors as mcolors

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 20
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 10309 samples in 0.005s...
[t-SNE] Computed neighbors for 10309 samples in 0.610s...
[t-SNE] Computed conditional probabilities for sample 1000 / 10309
[t-SNE] Computed conditional probabilities for sample 2000 / 10309
[t-SNE] Computed conditional probabilities for sample 3000 / 10309
[t-SNE] Computed conditional probabilities for sample 4000 / 10309
[t-SNE] Computed conditional probabilities for sample 5000 / 10309
[t-SNE] Computed conditional probabilities for sample 6000 / 10309
[t-SNE] Computed conditional probabilities for sample 7000 / 10309
[t-SNE] Computed conditional probabilities for sample 8000 / 10309
[t-SNE] Computed conditional probabilities for sample 9000 / 10309
[t-SNE] Computed conditional probabilities for sample 10000 / 10309
[t-SNE] Computed conditional probabilities for sample 10309 / 10309
[t-SNE] Mean sigma: 0.040590
[t-SNE] KL divergence after 250 iterations with early exaggeration: 79.295052
[t-SNE] KL 

Loading BokehJS ...